### Three minutes TVB simulation. Response to Dr. Solodkin's request

1. use the tvb 3 mins simulation results to replace the 1 sec simuation. 
2. investigate the EPAD data base
3. 01/13/2022 lab meeting report, the statistical difference between 1 sec simulation and 3 mins simulation

In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/home/yat-lok/workspace/tvbtools/')
from tools import concat, statools, signaltools
import matplotlib.pyplot as plt

In [2]:
prefix_filename = '/home/yat-lok/workspace/data4project/gc3mins/'
amp = prefix_filename + 'ampres.xlsx'
freq = prefix_filename + 'freqres.xlsx'
delay = prefix_filename + 'delayres.xlsx'
amp = concat.LI_cal(amp, [3,5], ('LI_amp_gamma', 'LI_amp_theta'))
freq = concat.LI_cal(freq, [3,5], ('LI_freq_gamma', 'LI_freq_theta'))
delay = pd.read_excel(delay)

la3mins = amp.iloc[:,:2]
tmp_la = pd.concat([amp.iloc[:,-2:], freq.iloc[:,-2:]], axis=1)
la3mins_table = pd.concat([la3mins, tmp_la], axis=1)
outdir = prefix_filename + 'la3mins.xlsx'
# la3mins_table.to_excel(outdir)

#### Second part, permutation value in tvb3mins metrics


In [3]:
raw_tvb = pd.concat([amp.iloc[:,:-2], freq.iloc[:,2:-2], delay.iloc[:,2:]], axis=1)
print(raw_tvb)

   group caseid  amp_gamma_left  amp_gamma_right  amp_theta_right  \
0    SNC  2820A        2.562834         2.507292         1.334379   
1    SNC  3168A        2.432937         2.320258         1.321384   
2    SNC  3358A        0.000000         2.570389         1.207173   
3    SNC  3610A        2.163050         2.385104         1.330792   
4    SNC  4073A        2.216265         2.603418         1.351165   
..   ...    ...             ...              ...              ...   
69    AD  3255A        2.438360         2.559287         1.355220   
70    AD  4612A        0.000000         2.532911         1.335439   
71    AD  5368A        2.186795         2.599101         1.355143   
72    AD  5571A        0.000000         2.573279         1.275507   
73    AD  7673A        2.604409         2.437727         1.345592   

    amp_theta_left  freq_gamma_left  freq_gamma_right  freq_theta_left  \
0         1.126187             1378              5616              879   
1         0.945234     

In [4]:
raw_tvb_stats = statools.stats_calculator(raw_tvb, tails='greater', n=3)
# print(statools.stats_calculator(la3mins_table, tails='greater'))

# comparison to 1 sec simulation
gc1sec_table = pd.read_excel('/home/yat-lok/workspace/data4project/lateralization/gc1sec_res/gc1sec_stats.xlsx')
print(raw_tvb_stats)
print(gc1sec_table)

                  (SNC, NC)  (SNC, MCI)  (SNC, AD)  (NC, MCI)  \
amp_gamma_left        0.612      0.3264     0.2850     0.2850   
amp_gamma_right       0.992      0.9920     0.9920     0.9920   
amp_theta_right       0.781      0.6696     0.6696     0.6696   
amp_theta_left        0.785      0.4548     0.2600     0.2600   
freq_gamma_left       0.803      0.6108     0.1845     0.1845   
freq_gamma_right      0.890      0.5700     0.2925     0.1440   
freq_theta_left       0.772      0.5016     0.1560     0.1845   
freq_theta_right      0.940      0.5730     0.5730     0.1350   
pcg_delay             0.999      0.3360     0.9990     0.3360   

                               (NC, AD)  (MCI, AD)  
amp_gamma_left                    0.285     0.2850  
amp_gamma_right                   0.992     0.9920  
amp_theta_right                  0.6696     0.6696  
amp_theta_left                    0.126     0.2700  
freq_gamma_left                   0.054     0.1230  
freq_gamma_right  0.03600000000

#### 3. the statistical diff between 3mins sim and 1sec sim

In [21]:

print(np.array(gc1sec_table.iloc[:,4+1].astype(str).str.extract(r'(\d+.\d+)').astype(float)).flatten())
for i in range(6):
    x1 = np.array(raw_tvb_stats.iloc[:8,i].astype(str).str.extract(r'(\d+.\d+)').astype(float)).flatten()
    x2 = np.array(gc1sec_table.iloc[:8, i+1].astype(str).str.extract(r'(\d+.\d+)').astype(float)).flatten()
    p_value = statools.permutation_test(x1, x2, iteration=1000, tails="twosides")
    print(f"p_value of the {raw_tvb_stats.columns[i]} is {p_value}")

[0.014 0.008 0.031 0.445 0.029 0.941 0.03  0.504 0.008 0.439 0.071 0.118
 0.062]
p_value of the ('SNC', 'NC') is 0.005994005994005994
p_value of the ('SNC', 'MCI') is 0.6893106893106893
p_value of the ('SNC', 'AD') is 0.3206793206793207
p_value of the ('NC', 'MCI') is 0.935064935064935
p_value of the ('NC', 'AD') is 0.7952047952047953
p_value of the ('MCI', 'AD') is 0.9300699300699301


In [ ]:
for i in raw_tvb.columns[2:]:
    signaltools.SignalToolkit.psvplot(x='group', y=i, df=raw_tvb)
    plt.show()
for i in la3mins_table.columns[2:]:
    signaltools.SignalToolkit.psvplot(x='group', y=i, df=la3mins_table)
    plt.show()